# IMPORTS

In [2]:
import numpy as np
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
import pickle
from multiprocessing import Process
from multiprocessing import Pool
import os
import pandas as pd
from gensim.models import Word2Vec
from nltk import TweetTokenizer
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from gensim.models.keyedvectors import KeyedVectors
from lda import LDA

# DATA RETRIEVAL

In [3]:
#string to number for class
dic_aggression_level = {
    'NAG' : 1,
    'CAG' : 2,
    'OAG' : 3
}

#train data
train_pd = shuffle(pd.concat((pd.read_csv("train.csv")[['Data', 'Label']], pd.read_csv("valid.csv")[['Data', 'Label']])), random_state=20)
train_pd['Label_num'] = train_pd.Label.map(dic_aggression_level)

#test data
test_fb_pd = shuffle(pd.read_csv("test_fb.csv")[['Data', 'Label']], random_state=20)
test_fb_pd['Label_num'] = test_fb_pd.Label.map(dic_aggression_level)
test_tw_pd = shuffle(pd.read_csv("test_tw.csv")[['Data', 'Label']], random_state=20)
test_tw_pd['Label_num'] = test_tw_pd.Label.map(dic_aggression_level)

#tokenizer for word2vec
tokenizer = TweetTokenizer()
def create_tokens(x):
    return tokenizer.tokenize(x.lower())

# word2vec_model = Word2Vec(list(map(create_tokens, train_pd['Data'])), size=100, window=5, min_count=3, workers=50)
# glove_model = KeyedVectors.load_word2vec_format("gensim_glove_vectors.txt", binary=False)
word2vec_model = KeyedVectors.load("glove.twitter.27B/glove.twitter.27B.200d.txt.vector.bin")
vocabulary = set(word2vec_model.wv.vocab.keys())

tfidf_model = TfidfVectorizer(vocabulary=vocabulary, tokenizer=create_tokens)
tfidf_model.fit(train_pd['Data'])
tfidf_values = tfidf_model.transform([" ".join(vocabulary)]).toarray()[0]

/opt/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:26: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


# GENERATE DATA MATRIX

In [4]:
def vectorise_sentence(sentence, method='avg'):
    global tokenizer
    tokens = tokenizer.tokenize(sentence)
    #addition
    if method=='add':
        vector = np.array([0]*word2vec_model['hate'].shape[0])
        for token in tokens:
            if token in vocabulary:
                vector = vector + word2vec_model[token]
        return vector 
    #average
    if method=='avg':
        vector = np.array([0]*word2vec_model['hate'].shape[0])
        count = 0
        for token in tokens:
            if token in vocabulary:
                vector = vector + word2vec_model[token]
                count += 1
        if count != 0:
            vector = vector / count
        return vector 
    #Average of Word2Vec vectors with TF-IDF : this is one of the best approach which I will recommend. 
    #Just take the word vectors and multiply it with their TF-IDF scores. Just take the average and it 
    #will represent your sentence vector.
    if method=='tfidf':
        vector = np.array([0]*word2vec_model['hate'].shape[0])
        count = 0
        for token in tokens:
            if token in vocabulary:
                vector = vector + (word2vec_model[token]*tfidf_values[tfidf_model.vocabulary_[token]])
                count += tfidf_values[tfidf_model.vocabulary_[token]]
        if count != 0:
            vector = vector / count
        return vector

method = 'tfidf'
X_train = np.array([vectorise_sentence(sentence, method) for sentence in train_pd['Data']])
y_train = np.array(train_pd['Label_num'])
X_fb_test = np.array([vectorise_sentence(sentence, method) for sentence in test_fb_pd['Data']])
y_fb_test = np.array(test_fb_pd['Label_num'])
X_tw_test = np.array([vectorise_sentence(sentence, method) for sentence in test_tw_pd['Data']])
y_tw_test = np.array(test_tw_pd['Label_num'])
print(X_train.shape, y_train.shape, X_fb_test.shape, y_fb_test.shape, X_tw_test.shape, y_tw_test.shape)

(15001, 200) (15001,) (916, 200) (916,) (1257, 200) (1257,)


# POS TAGGING

In [5]:
%%time
from multiprocessing import Pool

#tokenize
from nltk.tokenize import TweetTokenizer
tweeter = TweetTokenizer()
process_pool = Pool(processes=55)
pos_train = list(process_pool.map(tweeter.tokenize, train_pd['Data']))
pos_fb_test  = list(process_pool.map(tweeter.tokenize, test_fb_pd['Data']))
pos_tw_test  = list(process_pool.map(tweeter.tokenize, test_tw_pd['Data']))
process_pool.close()
print("Finished tokenising")

#remove stop words
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
def f(x):
    return tuple(filter(lambda y: y not in stop_words, x))
process_pool = Pool(processes=55)
pos_train = list(process_pool.map(f, pos_train))
pos_fb_test = list(process_pool.map(f , pos_fb_test))
pos_tw_test = list(process_pool.map(f , pos_tw_test))
process_pool.close()
print("Finished removing stop words")

#stemming
from nltk.stem import PorterStemmer
ps = PorterStemmer()
def f(x):
    return tuple(map(ps.stem, x))
process_pool = Pool(processes=55)
pos_train = list(process_pool.map(f, pos_train))
pos_fb_test = list(process_pool.map(f , pos_fb_test))
pos_tw_test = list(process_pool.map(f , pos_tw_test))
process_pool.close()
print("Finished stemming")

#POS tagging
from nltk import pos_tag
process_pool = Pool(processes=55)
pos_train = list(process_pool.map(pos_tag, pos_train))
pos_fb_test = list(process_pool.map(pos_tag, pos_fb_test))
pos_tw_test = list(process_pool.map(pos_tag, pos_tw_test))
process_pool.close()
print("Finished POS tagging")

#recombination
# def f(x): #pos concatenation
#     return " ".join(list(map("".join, x)))

def f(x): #pos only sentences
    return " ".join(list(map(lambda y: y[1], x)))

process_pool = Pool(processes=55)
pos_train = list(process_pool.map(f, pos_train))
pos_fb_test = list(process_pool.map(f, pos_fb_test))
pos_tw_test = list(process_pool.map(f, pos_tw_test))
process_pool.close()
print("Finished recombining")


#create tfidf matrix
tfidf = TfidfVectorizer()
pos_train = tfidf.fit_transform(pos_train)
pos_fb_test = tfidf.transform(pos_fb_test)
pos_tw_test = tfidf.transform(pos_tw_test)

# print(pos_train[:10], pos_test[:10])
# print(pos_train.toarray().shape)

X_train = np.hstack((X_train, pos_train.toarray()))
X_fb_test = np.hstack((X_fb_test, pos_fb_test.toarray()))
X_tw_test = np.hstack((X_tw_test, pos_tw_test.toarray()))
print(X_train.shape, y_train.shape, X_fb_test.shape, y_fb_test.shape, X_tw_test.shape, y_tw_test.shape)

Finished tokenising
Finished removing stop words
Finished stemming
Finished POS tagging
Finished recombining
(15001, 233) (15001,) (916, 233) (916,) (1257, 233) (1257,)
CPU times: user 1.55 s, sys: 5.96 s, total: 7.5 s
Wall time: 9.17 s
Parser   : 168 ms


# LDA

In [6]:
%%time
topics = 150
lda_vect = LDA(n_topics=topics, n_iter=100)
count_vect = CountVectorizer()

count_train = count_vect.fit_transform(train_pd['Data'])
lda_train = lda_vect.fit_transform(count_train)
X_train = np.hstack((X_train, lda_train))
X_fb_test = np.hstack((X_fb_test, lda_vect.transform(count_vect.transform(test_fb_pd['Data']))))
X_tw_test = np.hstack((X_tw_test, lda_vect.transform(count_vect.transform(test_tw_pd['Data']))))

print(X_train.shape, y_train.shape, X_fb_test.shape, y_fb_test.shape, X_tw_test.shape, y_tw_test.shape)

INFO:lda:n_documents: 15001
INFO:lda:vocab_size: 24374
INFO:lda:n_words: 354658
INFO:lda:n_topics: 150
INFO:lda:n_iter: 100
/home/Abhor/.local/lib/python3.5/site-packages/lda/utils.py:55: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if sparse and not np.issubdtype(doc_word.dtype, int):
INFO:lda:<0> log likelihood: -5043427
INFO:lda:<10> log likelihood: -3493953
INFO:lda:<20> log likelihood: -3323876
INFO:lda:<30> log likelihood: -3255379
INFO:lda:<40> log likelihood: -3215893
INFO:lda:<50> log likelihood: -3192165
INFO:lda:<60> log likelihood: -3174183
INFO:lda:<70> log likelihood: -3158642
INFO:lda:<80> log likelihood: -3150066
INFO:lda:<90> log likelihood: -3142863
INFO:lda:<99> log likelihood: -3137464


(15001, 383) (15001,) (916, 383) (916,) (1257, 383) (1257,)
CPU times: user 49.3 s, sys: 32 ms, total: 49.3 s
Wall time: 49.3 s


NN :=
---

In [9]:
max_num = X_train.shape[1]
# hidden_layer_sizes = [(max_num//2,), (max_num,), (max_num//2,max_num//2), (max_num,max_num//2), 
#                       (max_num//2, max_num//2, max_num//2), (max_num//2, max_num, max_num//2)]
hidden_layer_sizes = [(max_num//2, max_num//2, max_num//2, max_num//2),
                     (max_num//2, max_num, max_num, max_num//2)]
activation = ['tanh', 'relu']
solver = ['sgd', 'adam']
alpha = [10**(-i) for i in range(1, 6, 2)]
batch_size = [150, 250]
learning_rate = ['adaptive']
learning_rate_init = [10**(-i) for i in range(1, 6, 2)]
max_iter = [2000]

In [10]:
NN_list = []
for a in max_iter:
    for b in learning_rate_init:
        for c in learning_rate:
            for d in batch_size:
                for e in alpha:
                    for f in solver:
                        for g in activation:
                            for h in hidden_layer_sizes:
                                
                                NN_list.append(
                                        MLPClassifier(
                                                        hidden_layer_sizes=h,
                                                        activation = g,
                                                        solver = f,
                                                        alpha = e,
                                                        batch_size = d,
                                                        learning_rate = c,
                                                        learning_rate_init = b,
                                                        max_iter = a,
                                                        early_stopping=True
                                                )                                                    
                                )

no_of_models = len(NN_list)
print("Number of models:", no_of_models)

Number of models: 144


In [ ]:
%%time
# run estimators
pooly = Pool(processes = 45)
save_folder = "results/NN_word2vec/"


def run_and_dump_estimator(estimator):
    %time estimator.fit(X_train, y_train)
    y_fb_pred = estimator.predict(X_fb_test)
    y_tw_pred = estimator.predict(X_tw_test)
    accuracy_fb = accuracy_score(y_fb_test, y_fb_pred)
    accuracy_tw = accuracy_score(y_tw_test, y_tw_pred)
    f1_fb = f1_score(y_fb_test, y_fb_pred, average='weighted')
    f1_tw = f1_score(y_tw_test, y_tw_pred, average='weighted')
    confusion_fb = confusion_matrix(y_fb_test, y_fb_pred)
    confusion_tw = confusion_matrix(y_tw_test, y_tw_pred)
    pickle.dump({
        'estimator': estimator,
        "accuracy": [accuracy_fb, accuracy_tw],
        'f1': [f1_fb, f1_tw],
        'confusion': [confusion_fb, confusion_tw]
    }, open(os.path.join(save_folder, str(f1_fb).split('.')[1]+'.pickle'), 'wb'))

pooly.map_async(run_and_dump_estimator, NN_list)
# run_and_dump_estimator(NN_list[0])

pooly.close()
pooly.join()

open(os.path.join(save_folder, "9.pickle"), 'wb').close()

Process ForkPoolWorker-288:
Process ForkPoolWorker-278:
Process ForkPoolWorker-279:
Process ForkPoolWorker-276:
Process ForkPoolWorker-281:
Process ForkPoolWorker-289:
Process ForkPoolWorker-295:
Process ForkPoolWorker-308:
Process ForkPoolWorker-286:
Process ForkPoolWorker-297:
Process ForkPoolWorker-298:
Process ForkPoolWorker-296:
Process ForkPoolWorker-294:
Process ForkPoolWorker-301:
Process ForkPoolWorker-300:
Traceback (most recent call last):
Process ForkPoolWorker-306:
Process ForkPoolWorker-311:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-303:
Process ForkPoolWorker-310:
Process ForkPoolWorker-307:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-302:
Process ForkPoolWorker-314:
Process ForkPoolWorker-304:
Process ForkPoolWorker-305:
Process ForkPoolWorker-313:
Process ForkPoolWorker-299:
Traceback (most recent call last):
Process ForkPoolWorker-315:
Process ForkPoolWorker-309:
  File

  File "/opt/anaconda/lib/python3.5/multiprocessing/queues.py", line 337, in get
    return ForkingPickler.loads(res)
AttributeError: Can't get attribute 'run_and_dump_estimator' on <module '__main__'>
  File "/opt/anaconda/lib/python3.5/multiprocessing/queues.py", line 337, in get
    return ForkingPickler.loads(res)
AttributeError: Can't get attribute 'run_and_dump_estimator' on <module '__main__'>
  File "/opt/anaconda/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/opt/anaconda/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/opt/anaconda/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
Traceback (most recent call last):
  File "/opt/anaconda/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/opt/anaconda/lib/python3

AttributeError: Can't get attribute 'run_and_dump_estimator' on <module '__main__'>
  File "/opt/anaconda/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda/lib/python3.5/multiprocessing/queues.py", line 337, in get
    return ForkingPickler.loads(res)
Process ForkPoolWorker-292:
  File "/opt/anaconda/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/opt/anaconda/lib/python3.5/multiprocessing/queues.py", line 337, in get
    return ForkingPickler.loads(res)
  File "/opt/anaconda/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/opt/anaconda/lib/python3.5/multiprocessing/queues.py", line 337, in get
    return ForkingPickler.loads(res)
  File "/opt/anaconda/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda/lib/python3.5/multiprocessing/process.py", line 93, in run
 

AttributeError: Can't get attribute 'run_and_dump_estimator' on <module '__main__'>
AttributeError: Can't get attribute 'run_and_dump_estimator' on <module '__main__'>
AttributeError: Can't get attribute 'run_and_dump_estimator' on <module '__main__'>
Traceback (most recent call last):
  File "/opt/anaconda/lib/python3.5/multiprocessing/queues.py", line 337, in get
    return ForkingPickler.loads(res)
  File "/opt/anaconda/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/opt/anaconda/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
AttributeError: Can't get attribute 'run_and_dump_estimator' on <module '__main__'>
  File "/opt/anaconda/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/opt/anaconda/lib/python3.5/multiprocessing/process.py", line 93

CPU times: user 6.28 s, sys: 124 ms, total: 6.41 s
Wall time: 1min 8s


...

CPU times: user 4min 21s, sys: 36min 26s, total: 40min 48s
Wall time: 17min 35s


/home/Abhor/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


CPU times: user 6min 8s, sys: 52min 49s, total: 58min 58s
Wall time: 20min 25s
CPU times: user 6min 34s, sys: 56min 20s, total: 1h 2min 54s
Wall time: 32min 8s
CPU times: user 6min 14s, sys: 53min 32s, total: 59min 47s
Wall time: 20min 16s
CPU times: user 5min 14s, sys: 42min 8s, total: 47min 23s
Wall time: 27min 30s
CPU times: user 9min 13s, sys: 1h 20min 33s, total: 1h 29min 47s
Wall time: 34min 54s
CPU times: user 9min 37s, sys: 1h 22min 52s, total: 1h 32min 30s
Wall time: 35min 6s
CPU times: user 6min 49s, sys: 1h 26s, total: 1h 7min 16s
Wall time: 23min 25s
CPU times: user 5min 57s, sys: 51min 44s, total: 57min 41s
Wall time: 28min 13s


/home/Abhor/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


CPU times: user 9min 37s, sys: 1h 25min 14s, total: 1h 34min 51s
Wall time: 35min 21s
CPU times: user 7min 43s, sys: 1h 6min 12s, total: 1h 13min 56s
Wall time: 23min 3s
CPU times: user 7min 12s, sys: 1h 47s, total: 1h 7min 59s
Wall time: 35min 38s
CPU times: user 8min 24s, sys: 1h 14min 31s, total: 1h 22min 56s
Wall time: 18min 53s
CPU times: user 7min 20s, sys: 1h 5min 47s, total: 1h 13min 7s
Wall time: 35min 50s
